<a href="https://colab.research.google.com/github/XiaochuanAi/sample.github.io/blob/main/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [ ]:
!bash ./datasets/download_cyclegan_dataset.sh horse2zebra

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_cyclegan_model.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
!bash ./scripts/download_cyclegan_model.sh horse2zebra

# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [ ]:
!python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan --display_id -1 lr 0.0002

In [2]:
!python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan --display_id -1 --n_epochs 20 --n_epochs_decay 10 --lr 0.0002   --load_size 128 --crop_size 128


python3: can't open file '/content/train.py': [Errno 2] No such file or directory


# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
!python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_real.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt
import os
import random
random.seed(1)

# Define the directory containing the images
image_dir = './results/horse2zebra_pretrained/test_latest/images/'

# Get all fake image filenames
all_fake_images = [f for f in os.listdir(image_dir) if f.endswith('_fake.png')]

# Randomly select 8 fake images
selected_fake_images = random.sample(all_fake_images, 8)

# Set up the figure with 2 rows of 2x4 subplots (one row for fake, one for real)
fig, axes = plt.subplots(2, 4, figsize=(12, 6))

# First row: fake/generated images
for i, ax in enumerate(axes[0]):
    fake_path = os.path.join(image_dir, selected_fake_images[i])
    img = plt.imread(fake_path)
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f'Fake: {selected_fake_images[i]}', fontsize=8)

# Second row: corresponding real/original images
for i, ax in enumerate(axes[1]):
    real_filename = selected_fake_images[i].replace('_fake.png', '_real.png')
    real_path = os.path.join(image_dir, real_filename)
    if os.path.exists(real_path):
        img = plt.imread(real_path)
        ax.imshow(img)
    else:
        ax.text(0.5, 0.5, 'Missing', ha='center', va='center')
    ax.axis('off')
    ax.set_title(f'Real: {real_filename}', fontsize=8)

plt.tight_layout
plt.savefig('/kaggle/working/output_plot.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
!pip install pytorch-fid
import os
import shutil


# Paths
generated_dir = './results/horse2zebra_pretrained/test_latest/images/'
real_dir = './datasets/horse2zebra/testB/'

# Temporary folders for FID computation
fid_fake_dir = './fid_temp/fake'
fid_real_dir = './fid_temp/real'

# Ensure clean folders
for path in [fid_fake_dir, fid_real_dir]:
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

# Copy fake images
for file in os.listdir(generated_dir):
    if file.endswith('_fake.png'):
        shutil.copy(os.path.join(generated_dir, file), os.path.join(fid_fake_dir, file))

# Copy real images
for file in os.listdir(real_dir):
    if file.endswith('.jpg') or file.endswith('.png'):
        shutil.copy(os.path.join(real_dir, file), os.path.join(fid_real_dir, file))

import subprocess

result = subprocess.run(
    ['python', '-m', 'pytorch_fid', fid_real_dir, fid_fake_dir],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Print the actual FID score
print(result.stdout)